In [ ]:
import pandas as pd

In [ ]:

    
    %%javascript
    IPython.OutputArea.prototype._should_scroll = function(lines) {
        return false;
    }
    
    

In [ ]:
dataFinal = pd.read_csv('../sites/USxNQtempregression3row.csv')

In [ ]:

    
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from IPython.display import HTML
import io
import base64
from scipy.spatial import distance
from scipy import stats
import os
import nbformat as nbf
import math


if 'Time' in dataFinal.columns:
    dataFinal['Time'] = pd.to_datetime(dataFinal['Time'])
    dataFinal.set_index('Time', inplace=True)
    dataFinal['Year'] = dataFinal.index.year

unique_years = dataFinal['Year'].unique()

for year in unique_years:
    data_subset = dataFinal.loc[dataFinal['Year'] == year].copy()

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    axs[0].plot(data_subset.index, data_subset['Daily_NEE'], label='Daily_NEE')
    axs[0].plot(data_subset.index, data_subset['Daily_RECO_temp'], label='Daily_RECO_temp')
    axs[0].scatter(data_subset.index, data_subset['predicted_RECO'], color='red', label='predicted_RECO')
    axs[0].set_title(f'Daily_NEE & Daily_RECO_temp over Time for year {year}')
    axs[0].legend()

    axs[1].plot(data_subset.index, data_subset['Daily_EF'], label='Daily_EF')
    axs[1].bar(data_subset.index, data_subset['Daily_P'], color = 'red', label='Daily_P')
    axs[1].set_title(f'Daily_EF & Daily_P over Time for year {year}')
    axs[1].legend()

    plt.ylabel('Values')
    plt.xlabel('Time')

    plt.tight_layout()
    plt.show()

    recoDf = dataFinal[dataFinal['predicted_RECO'].notnull()]
    recoDf['RecoMinusNee'] = recoDf['predicted_RECO'] - recoDf['Daily_NEE']
    recoDf['RecoMinusRecotemp'] = recoDf['predicted_RECO'] - recoDf['Daily_RECO_temp']
    recoDf.head(10)


# Press Cell - Current Output - Toggle Output to disable scorrling output of graph. 

layerOne = recoDf.copy()
layerOne['DailyP_Zero_F1'] = np.where((layerOne['Daily_P'] == 0) , 'Drop', 'Take')


layerTwo = layerOne.copy()
layerTwo['Daily_NEE_Change'] = layerTwo['Daily_NEE'].diff(1)


layerThree = layerTwo.copy()
layerThree['Daily_RECO_temp_Change'] = layerThree['Daily_RECO_temp'].diff(1)


layerFour = layerThree.copy()
epsilon = math.pi/100
def compare_values(nee_change, reco_temp_change):
    if abs(nee_change - reco_temp_change) <= epsilon:
        return "Drop"
    else:
        return "Take"
layerFour['Slope_NEE_RECO_F2'] = layerFour.apply(lambda row: compare_values(row['Daily_NEE_Change'], row['Daily_RECO_temp_Change']), axis=1)


layerFive = layerFour.copy()
layerFive['Daily_EF_Change'] = layerFive['Daily_EF'].diff(1)
layerFive['Daily_EF_Neg_F3'] = np.where((layerFive['Daily_EF_Change'] < 0) , 'Drop', 'Take')


layerSix = layerFive.copy()
def check_drop(row):
    count = 0
    for col in ['DailyP_Zero_F1', 'Slope_NEE_RECO_F2', 'Daily_EF_Neg_F3']:
        if row[col] == 'Drop':
            count += 1
        if count >= 2:
            return 'Drop'
    return 'Take'
layerSix['FinalDrop'] = layerSix.apply(check_drop, axis=1)


layerSeven = layerSix.copy()
# First, convert 'Time' from index to column
layerSeven.reset_index(level=0, inplace=True)
# Then create a new index starting from 1
layerSeven.index = layerSeven.index + 1
years = layerSeven['Year'].unique()  # List of unique years
layerSeven.rename(columns={'Unnamed: 0':'Original_Index'}, inplace=True)
# create an empty dictionary to store each dataframe
dfs = {}
# Using a for loop to create a dataframe for each unique year
for year in years:
    dfs["df_{}".format(year)] = layerSeven[layerSeven['Year'] == year]
# For example
df_2010 = dfs["df_2010"]
df_2010.head(28)
# the left most column (layerSeven index without name on top) is the red points index in a year (predicted_Reco)
# the Original_Index is the index of that point from the dataFinal (original data frame)


# merge dataFinal and layerSevens on 'Unnamed: 0' column
merged_df = pd.merge(dataFinal, layerSeven, how='inner', left_on='Unnamed: 0', right_on='Original_Index')
# filter rows where FinalDrop is Drop
filtered_df = merged_df.query('FinalDrop != "Drop"')
filtered_df = filtered_df.rename(columns = {'Unnamed: 0':'Index_Origin'})
filtered_df.index = np.arange(1, len(filtered_df) + 1)
  
        

In [ ]:

# After Filter

new_dataFrame = layerSeven[layerSeven['FinalDrop'] == 'Drop'][['Original_Index']]

filtered_df = dataFinal.copy()
for index, row in filtered_df.iterrows():
    if row['Unnamed: 0'] in new_dataFrame['Original_Index'].tolist():
        filtered_df  = filtered_df .drop(index)
         
if 'Time' in filtered_df.columns:
    filtered_df['Time'] = pd.to_datetime(filtered_df['Time'])
    filtered_df.set_index('Time', inplace=True)
    filtered_df['Year'] = filtered_df.index.year

unique_years_final = filtered_df['Year'].unique()

for year in unique_years_final:
    data_subset_final = filtered_df.loc[filtered_df['Year'] == year].copy()

    # Plotting
    fig, axs = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    axs[0].plot(data_subset_final.index, data_subset_final['Daily_NEE'], label='Daily_NEE')
    axs[0].plot(data_subset_final.index, data_subset_final['Daily_RECO_temp'], label='Daily_RECO_temp')
    axs[0].scatter(data_subset_final.index, data_subset_final['predicted_RECO'], color='red', label='predicted_RECO')
    axs[0].set_title(f'Daily_NEE & Daily_RECO_temp over Time for year {year}')
    axs[0].legend()

    axs[1].plot(data_subset_final.index, data_subset_final['Daily_EF'], label='Daily_EF')
    axs[1].bar(data_subset_final.index, data_subset_final['Daily_P'], color = 'red', label='Daily_P')
    axs[1].set_title(f'Daily_EF & Daily_P over Time for year {year}')
    axs[1].legend()
    
    plt.ylabel('Values')
    plt.xlabel('Time')
    plt.tight_layout()
    plt.show()
    
    #to get droped points index, put this: 
    #droppedDF = layerSeven[layerSeven['FinalDrop'] == 'Drop']
    #droppedDF.head()
        